In [1]:
##
#Use Tweepy(search_users API) to search users for the keywords [sol,eth,btc,algo]
#This API doesnt do an exact match, it sometimes returns irrelevant usernames
#Each user's followers_count and description are fetched from the API response
#This API fetches maximum of 1000 records per call
##

import tweepy
import pandas as pd

consumer_key = "ZkoxSlIeJAA1D1IAOMDDjU86r"
consumer_secret = "YQGPWddleCIq5pl7WtryUdGAZ8Cb7OuubFmbwrr9fIHLzzDjy7"
access_token = "1328488124155121665-gxrvcEBNfahpvbel3cIX9OLBgiOxjU"
access_token_secret = "sTVJNm0BQ7mRrTQaobH1XNv9yEB5lymLINXV6aVBWkLf6"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

search_list = ['sol','eth','btc','algo']
twitter_username = []
username_followers = []
username_bio = []
user_type = []
for username in search_list: 
    for user in tweepy.Cursor(api.search_users,q = username).items(1000):
        twitter_username.append(user.screen_name)
        username_followers.append(user.followers_count)
        username_bio.append(user.description)
        user_type.append(username)

In [2]:
#Create DATAFRAME from the response arrays

df = pd.DataFrame({'username': twitter_username, "followercount" : username_followers,"Bio" : username_bio,"Type" : user_type})
df.shape

(4000, 4)

In [3]:
#use regex to create list of usernames ending with [sol,eth,btc,algo] from the response

import re
regex_check = ['([a-zA-Z0-9_]{1,11}sol$)','([a-zA-Z0-9_]{1,11}eth$)','([a-zA-Z0-9_]{1,11}btc$)','([a-zA-Z0-9_]{1,11}algo$)']
matched_username = []

for item in regex_check:    
    matching_username = list(filter(lambda x: re.match(item, x), twitter_username))
    matched_username.append(matching_username)

username_flatlist = [x for xs in matched_username for x in xs]

In [4]:
#filter dataframe username column based on the regex matched usernames and write it to a csv file

df_new = df[df['username'].isin(username_flatlist)]
df_new = df_new.sort_values(['Type','followercount'],ascending=False).groupby('Type').head(100)
df_new.to_csv('twitter_username.csv')